In [5]:
import argparse
import logging
import random
import time

import numpy as np
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import AdaLoraConfig, TaskType, get_peft_model, PeftModel
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer, get_scheduler
from utils import (
    compute_kl,
    create_lotr_dataloader_from_dataset,
    create_bookcorpse_dataloader,
    get_answer_loss,
    get_rand_ans_loss,
)

import args
import config

In [2]:
torch.manual_seed(8888)
np.random.seed(8888)
random.seed(8888)

In [7]:
prompt = "The hobbits were so suprised seeing their friend"

## Load model

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
model = AutoModelForCausalLM.from_pretrained(config.model_name, return_dict=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
# Load the Lora model
model = PeftModel.from_pretrained(model, "/scratch/sa6981/llm_unlearn/finetune_copyright/models/opt1.3b_unlearned_1")

In [8]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [9]:
tokens = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.3,
            do_sample=True,
            top_p=0.95, top_k=50,
        )

In [10]:
print(tokenizer.decode(tokens[0]))

</s>The hobbits were so suprised seeing their friend.                                               ! !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


## create a function

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def generate(prompt: str, max_new_tokens: int, temperature: float, do_sample: bool) -> None:
        model = AutoModelForCausalLM.from_pretrained(config.model_name, return_dict=True, load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        # Load the Lora model
        model = PeftModel.from_pretrained(model, "/scratch/sa6981/llm_unlearn/finetune_copyright/models/opt1.3b_unlearned_1")

        # Generate text
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            top_p=0.95, top_k=50,
        )
        print(tokenizer.decode(tokens[0]))

In [12]:
prompt = "The President to US is"
generate(prompt, 256, 0.3, True)

</s>The President to US is not              !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [13]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def generate_after_trainable_params(prompt: str, max_new_tokens: int, temperature: float, do_sample: bool) -> None:
        # Import the model
        model = AutoModelForCausalLM.from_pretrained(config.model_name, return_dict=True, load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        # Load the Lora model
        model = PeftModel.from_pretrained(model, "/scratch/sa6981/llm_unlearn/finetune_copyright/models/opt1.3b_unlearned_tested")

        # Generate text
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            top_p=0.95, top_k=50,
        )
        print(tokenizer.decode(tokens[0]))

In [14]:
prompt = "The hobbits were so suprised seeing their friend"
generate_after_trainable_params(prompt, 256, 0.3, True)

</s>The hobbits were so suprised seeing their friend.                                                 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [15]:
prompt = "‘Not safe for ever,’ said Gandalf. ‘There are many things in the deep"
generate_after_trainable_params(prompt, 256, 0.3, True)

</s>‘Not safe for ever,’ said Gandalf. ‘There are many things in the deep and many.           ! !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [16]:
prompt = "Why have they come? Have you heard?’ asked Merry. He had now dressed"
generate_after_trainable_params(prompt, 256, 0.3, True)

</s>Why have they come? Have you heard?’ asked Merry. He had now dressed, and that they were all in danger.                                                                                                                                                                                                                                                       
